In [1]:
import pandas as pd
import re
import json
from rich import print

from utils import evaluate_ner, basic_post_processing


def parse_ner_output(generated_text, split_by=""):

    segments = generated_text.split("### Entities:" + split_by)
    if len(segments) < 2:
        return []

    after_entities = segments[1].strip()

    entities = []
    for line in after_entities.split("\n"):
        line = line.strip()
        match = re.match(r"^([A-Z]+):\s*(.*)", line)
        if match:
            label, entity_text = match.groups()
            if label in POSSIBLE_ENTITIES:
                entities.append({"label": label, "text": entity_text})

    return json.dumps(entities, ensure_ascii=False)


data_df = pd.read_csv("../data/test.csv")

BASE_PATH = "../results/sft/"
POSSIBLE_ENTITIES = [
    "PERS", "DOC", "QUANT", "ART", "TIME", "JOB",
    "MISC", "PCT", "ORG", "LOC", "PERIOD", "MON", "DATE"
]

In [2]:
files_to_eval = [
    "gemma-2-27b-it_16_ex.csv",
    "gemma-2-27b-it_32_ex.csv",
    "gemma-2-27b-it_64_ex.csv",
    "gemma-3-27b-it_16_ex.csv",
    "gemma-3-27b-it_32_ex.csv",
    "gemma-3-27b-it_64_ex.csv",
    "Qwen2.5-14B-Instruct_16_ex.csv",
    "Qwen2.5-14B-Instruct_32_ex.csv",
    "Qwen2.5-14B-Instruct_64_ex.csv",
]

phi_to_eval = [
    "Phi-4_16_ex.csv",
    "Phi-4_32_ex.csv",
    "Phi-4_64_ex.csv",
]

res = {}
for f_name in files_to_eval:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].apply(parse_ner_output)
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    res[f_name] = evaluate_ner(data_df)["ents_f"]
    
for f_name in phi_to_eval:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].map(lambda x: parse_ner_output(x, "assistant"))
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    res[f_name] = evaluate_ner(data_df)["ents_f"]
     
    
print(json.dumps(res, indent=2))

{
  "gemma-2-27b-it_16_ex.csv": 0.8653431481894565,
  "gemma-2-27b-it_32_ex.csv": 0.8603829160530191,
  "gemma-2-27b-it_64_ex.csv": 0.86366669137816,
  "gemma-3-27b-it_16_ex.csv": 0.8667719852553976,
  "gemma-3-27b-it_32_ex.csv": 0.8787969474786773,
  "gemma-3-27b-it_64_ex.csv": 0.8822429906542056,
  "Qwen2.5-14B-Instruct_16_ex.csv": 0.8512079476179725,
  "Qwen2.5-14B-Instruct_32_ex.csv": 0.8512751071992779,
  "Qwen2.5-14B-Instruct_64_ex.csv": 0.853515625,
  "Phi-4_16_ex.csv": 0.8694021453754408,
  "Phi-4_32_ex.csv": 0.8713910761154855,
  "Phi-4_64_ex.csv": 0.8742874287428744
}

In [5]:
best_files_to_eval = [
    "Qwen2.5-14B-Instruct_Best.csv",
    "gemma-2-27b-it_Best.csv",
    "gemma-3-27b-it_Best.csv",
]

best_res = {}
for f_name in best_files_to_eval:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].apply(parse_ner_output)
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    print(f_name)
    print(evaluate_ner(data_df))


for f_name in ["Phi-4_Best.csv"]:
    df = pd.read_csv(BASE_PATH + f_name)
    data_df["pred"] = df["generated_answer"].map(lambda x: parse_ner_output(x, "assistant"))
    data_df["pred"] = data_df.apply(basic_post_processing, axis=1)
    print(f_name)
    print(evaluate_ner(data_df))

Qwen2.5-14B-Instruct_Best.csv

{
    'ents_p': 0.8858895705521472,
    'ents_r': 0.8480399353986199,
    'ents_f': 0.8665516465381441,
    'ents_per_type': {
        'LOC': {'p': 0.899002493765586, 'r': 0.8814180929095354, 'f': 0.8901234567901234},
        'PCT': {'p': 1.0, 'r': 0.9555555555555556, 'f': 0.9772727272727273},
        'PERS': {'p': 0.95125284738041, 'r': 0.9586776859504132, 'f': 0.9549508346672766},
        'JOB': {'p': 0.7159763313609467, 'r': 0.5525114155251142, 'f': 0.6237113402061856},
        'PERIOD': {'p': 0.71875, 'r': 0.6216216216216216, 'f': 0.6666666666666667},
        'ORG': {'p': 0.9185336048879837, 'r': 0.8791423001949318, 'f': 0.898406374501992},
        'MISC': {'p': 0.416, 'r': 0.36619718309859156, 'f': 0.3895131086142322},
        'ART': {'p': 0.6964980544747081, 'r': 0.7552742616033755, 'f': 0.7246963562753036},
        'QUANT': {'p': 0.8518518518518519, 'r': 0.7752808988764045, 'f': 0.8117647058823529},
        'DATE': {'p': 0.8804123711340206, 'r': 0.8522954091816367, 'f': 0.8661257606490872},
        'TIME': {'p': 0.6, 'r': 0.3, 'f': 0.4},
        'DOC': {'p': 0.5483870967741935, 'r': 0.425, 'f': 0.47887323943661964},
        'MON': {'p': 0.9680511182108626, 'r': 0.9323076923076923, 'f': 0.9498432601880877}
    }
}

gemma-2-27b-it_Best.csv

{
    'ents_p': 0.902557856272838,
    'ents_r': 0.8703567758038467,
    'ents_f': 0.8861648852679572,
    'ents_per_type': {
        'LOC': {'p': 0.9064039408866995, 'r': 0.8997555012224939, 'f': 0.9030674846625768},
        'PCT': {'p': 1.0, 'r': 0.9888888888888889, 'f': 0.9944134078212291},
        'PERS': {'p': 0.9630642954856361, 'r': 0.9696969696969697, 'f': 0.9663692518874399},
        'JOB': {'p': 0.7686116700201208, 'r': 0.5814307458143074, 'f': 0.6620450606585788},
        'PERIOD': {'p': 0.7797619047619048, 'r': 0.7081081081081081, 'f': 0.7422096317280453},
        'ORG': {'p': 0.9282868525896414, 'r': 0.9083820662768031, 'f': 0.9182266009852216},
        'MISC': {'p': 0.34782608695652173, 'r': 0.28169014084507044, 'f': 0.311284046692607},
        'DOC': {'p': 0.6666666666666666, 'r': 0.45, 'f': 0.5373134328358209},
        'QUANT': {'p': 0.8735632183908046, 'r': 0.8539325842696629, 'f': 0.8636363636363635},
        'DATE': {'p': 0.9085365853658537, 'r': 0.8922155688622755, 'f': 0.9003021148036255},
        'ART': {'p': 0.7034220532319392, 'r': 0.7805907172995781, 'f': 0.7400000000000001},
        'TIME': {'p': 1.0, 'r': 0.7, 'f': 0.8235294117647058},
        'MON': {'p': 0.9839743589743589, 'r': 0.9446153846153846, 'f': 0.9638932496075352}
    }
}

gemma-3-27b-it_Best.csv

{
    'ents_p': 0.9052631578947369,
    'ents_r': 0.8712377037145793,
    'ents_f': 0.8879245847673201,
    'ents_per_type': {
        'LOC': {'p': 0.9388264669163545, 'r': 0.9193154034229829, 'f': 0.9289684990735022},
        'PCT': {'p': 1.0, 'r': 0.9777777777777777, 'f': 0.9887640449438202},
        'PERS': {'p': 0.9612932604735883, 'r': 0.9692378328741965, 'f': 0.965249199817101},
        'JOB': {'p': 0.7510204081632653, 'r': 0.5601217656012176, 'f': 0.6416739319965126},
        'PERIOD': {'p': 0.7597765363128491, 'r': 0.7351351351351352, 'f': 0.7472527472527473},
        'ORG': {'p': 0.9260954872465664, 'r': 0.9200779727095516, 'f': 0.9230769230769231},
        'MISC': {'p': 0.36363636363636365, 'r': 0.3380281690140845, 'f': 0.3503649635036496},
        'QUANT': {'p': 0.9176470588235294, 'r': 0.8764044943820225, 'f': 0.896551724137931},
        'DATE': {'p': 0.9146341463414634, 'r': 0.8982035928143712, 'f': 0.9063444108761329},
        'ART': {'p': 0.755868544600939, 'r': 0.679324894514768, 'f': 0.7155555555555556},
        'TIME': {'p': 0.6666666666666666, 'r': 0.6, 'f': 0.631578947368421},
        'DOC': {'p': 0.5588235294117647, 'r': 0.475, 'f': 0.5135135135135136},
        'MON': {'p': 0.9804560260586319, 'r': 0.9261538461538461, 'f': 0.9525316455696202}
    }
}

Phi-4_Best.csv

{
    'ents_p': 0.8976270155156678,
    'ents_r': 0.8663926002055499,
    'ents_f': 0.8817332835263354,
    'ents_per_type': {
        'LOC': {'p': 0.9100985221674877, 'r': 0.9034229828850856, 'f': 0.9067484662576687},
        'PCT': {'p': 0.9883720930232558, 'r': 0.9444444444444444, 'f': 0.9659090909090908},
        'PERS': {'p': 0.959349593495935, 'r': 0.9752066115702479, 'f': 0.9672131147540983},
        'JOB': {'p': 0.7203065134099617, 'r': 0.5722983257229832, 'f': 0.6378286683630195},
        'PERIOD': {'p': 0.75, 'r': 0.6810810810810811, 'f': 0.7138810198300283},
        'ORG': {'p': 0.9159553512803676, 'r': 0.9064327485380117, 'f': 0.9111691704768125},
        'DOC': {'p': 0.5517241379310345, 'r': 0.4, 'f': 0.463768115942029},
        'MISC': {'p': 0.39361702127659576, 'r': 0.2605633802816901, 'f': 0.3135593220338983},
        'ART': {'p': 0.7676348547717843, 'r': 0.7805907172995781, 'f': 0.7740585774058578},
        'QUANT': {'p': 0.8860759493670886, 'r': 0.7865168539325843, 'f': 0.8333333333333333},
        'DATE': {'p': 0.9008264462809917, 'r': 0.8702594810379242, 'f': 0.8852791878172589},
        'TIME': {'p': 1.0, 'r': 0.4, 'f': 0.5714285714285715},
        'MON': {'p': 0.9685534591194969, 'r': 0.9476923076923077, 'f': 0.9580093312597201}
    }
}